In [68]:
class ProgressBar:
	bar_string_fmt = "\rProgress: [{}{}] {:.2%} {}/{}"
	cnt = 0

	def __init__(self, total, bar_total=20):
		# task 的總數
		self.total = total

		# progress bar 的長度，可依個人喜好設定
		self.bar_total = bar_total

	def update(self, step=1):
		# 更新 progress bar 的進度

		total = self.total
		self.cnt += step

		# bar 的數量
		bar_cnt = (int((self.cnt/total)*self.bar_total))
		# 空白的數量
		space_cnt = self.bar_total - bar_cnt

		progress = self.bar_string_fmt.format(
			"#" * bar_cnt,
			" " * space_cnt,
			self.cnt/total,
			self.cnt,
			total
		)

		print(progress, end="")
		
		percent = self.cnt/total
		# 100%
		if percent == 1:
			print("\n")
		elif percent >= 1:
			print("")

In [3]:
# 載入需要的套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt
import tensorflow as tf
import re
import os

In [4]:
import requests
from bs4 import BeautifulSoup

# 發送 GET 請求獲取證券櫃檯買賣中心 (TWSE) 的網頁內容
url = "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2"
html = requests.get(url).text

# 使用 BeautifulSoup 解析網頁內容
soup = BeautifulSoup(html, "html.parser")

In [5]:
def saveToFile(category, stock):
	path = "csv/" + category + "/" + stock + ".csv"
	if not os.path.isfile(path):
		df = yf.download(stock, end = dt.date.today(), progress=False)
		df.to_csv(path)

In [46]:
# https://isin.twse.com.tw/isin/C_public.jsp?strMode=2
category_list = ["半導體業", "電機機械", "汽車工業", "化學工業", "生技醫療業", "鋼鐵工業"]

In [ ]:
for category in category_list:
	path = "./csv/" + category
	if not os.path.isdir(path): os.mkdir(path)

for tr in soup.find_all("tr")[2:]:
	text = tr.find(True).text
	if re.match("[0-9]{4}　.*", text):
		category = tr.select_one(':nth-child(5)').text
		if category in category_list:
			saveToFile(category, text[0:4] + ".TW")

In [44]:
# 取得道瓊指數
dji = yf.download("^DJI", end=dt.date.today())

def getDJI_LastData(date_str):
	date = dt.datetime.strptime(date_str, "%Y-%m-%d")
	one_day = dt.timedelta(days=1)

	while True:
		date -= one_day
		try:
			last = dji.loc[date]
			break
		except:
			pass

	return last

# data = dji.loc[date]

[*********************100%***********************]  1 of 1 completed


In [63]:
stock = "^DJI"
saveToFile("test", stock)
dji_open = np.loadtxt("csv/test/" + stock + ".csv", delimiter=",", skiprows=1, usecols=(1))
dji_open[-1-6:-1]

array([33175.390625  , 32999.19140625, 32906.16015625, 32873.46875   ,
       32656.36914062, 32780.96875   ])

In [73]:
days = 50

x_train = np.empty((0, 5 * days + 1))
y_train = np.empty((0, 5))

n = 0
for c in category_list:
	n += len(list(os.scandir("./csv/" + c)))
progress_bar = ProgressBar(n, bar_total=30)

for c in category_list:
	for file in os.scandir("./csv/" + c):
		data = np.loadtxt(file.path, delimiter=",", skiprows=1, usecols=(1,2,3,4,5))

		y = data[days:, :]
		size = y.shape[0]

		x = []

		n, m = data.shape
		for i in range(n - days):
			sub = data[i:i + days,:]
			x.append(np.concatenate(sub, axis=0))
		x = np.array(x).reshape(-1, 5 * days)
		x = np.concatenate((x, dji_open[-1 - size:-1].reshape(-1, 1)), axis=1)

		x_train = np.concatenate([x_train, x], axis=0)
		y_train = np.concatenate([y_train, y], axis=0)

		progress_bar.update()

Progress: [##############################] 100.00% 262/262



In [74]:
x_train.shape

(1023708, 251)

In [75]:
from keras.models import Sequential
from keras.layers import Dense
act = tf.keras.activations
opt = tf.keras.optimizers
los = tf.keras.losses

model = Sequential()

model.add(Dense(300, activation=act.relu))
model.add(Dense(100, activation=act.relu))
model.add(Dense(100, activation=act.relu))
model.add(Dense(80, activation=act.relu))
model.add(Dense(5, activation=act.linear))

model.compile(optimizer=opt.Adam(), loss=los.MeanAbsoluteError(), metrics=["accuracy"])

history = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
31991/31991 [==============================] - 112s 3ms/step - loss: 2.3992 - accuracy: 0.5366
Epoch 2/5
31991/31991 [==============================] - 109s 3ms/step - loss: 1.3937 - accuracy: 0.7004
Epoch 3/5
31991/31991 [==============================] - 112s 4ms/step - loss: 1.2556 - accuracy: 0.7369
Epoch 4/5
31991/31991 [==============================] - 110s 3ms/step - loss: 1.1897 - accuracy: 0.7456
Epoch 5/5
31991/31991 [==============================] - 122s 4ms/step - loss: 1.1450 - accuracy: 0.7526


In [81]:
stock = "2303.TW"
saveToFile("test", stock)
test_data = np.loadtxt("csv/test/" + stock + ".csv", delimiter=",", skiprows=1, usecols=(1,2,3,4,5))
x_test = []

n, m = test_data.shape
for i in range(n - days):
	sub = test_data[i:i + days,:]
	x_test.append(np.concatenate(sub, axis=0))
x_test = np.concatenate((np.array(x_test), dji_open[-1 - n + days:-1].reshape(-1, 1)), axis=1)

y_test = test_data[days:, :]

val_loss, val_acc = model.evaluate(x_test, y_test)
print(val_loss, val_acc)

177/177 [==============================] - 1s 3ms/step - loss: 0.3811 - accuracy: 0.8300
0.38113269209861755 0.8300248384475708


In [83]:
model.save("stock-predict.model")

INFO:tensorflow:Assets written to: stock-predict.model\assets


INFO:tensorflow:Assets written to: stock-predict.model\assets


In [ ]:
import yahoo_fin.stock_info as si
dow = si.tickers_other()
print(dow)

['A', 'AA', 'AAA', 'AAAU', 'AAC', 'AAC.U', 'AAC.W', 'AAIC', 'AAIC$B', 'AAIC$C', 'AAIN', 'AAM$A', 'AAM$B', 'AAMC', 'AAN', 'AAP', 'AAT', 'AAU', 'AB', 'ABB', 'ABBV', 'ABC', 'ABEQ', 'ABEV', 'ABG', 'ABM', 'ABR', 'ABR$D', 'ABR$E', 'ABR$F', 'ABT', 'AC', 'ACA', 'ACAQ', 'ACAQ.U', 'ACAQ.W', 'ACCO', 'ACEL', 'ACES', 'ACHR', 'ACHR.W', 'ACI', 'ACIO', 'ACM', 'ACN', 'ACP', 'ACP$A', 'ACR', 'ACR$C', 'ACR$D', 'ACRE', 'ACRO', 'ACRO.U', 'ACRO.W', 'ACSI', 'ACTV', 'ACU', 'ACV', 'ACVF', 'ACWV', 'ADC', 'ADC$A', 'ADCT', 'ADEX', 'ADEX.U', 'ADEX.W', 'ADFI', 'ADIV', 'ADM', 'ADME', 'ADNT', 'ADPV', 'ADRT', 'ADRT.U', 'ADT', 'ADX', 'AE', 'AEE', 'AEF', 'AEFC', 'AEG', 'AEL', 'AEL$A', 'AEL$B', 'AEM', 'AEMB', 'AENZ', 'AEO', 'AER', 'AES', 'AESC', 'AESR', 'AEVA', 'AEVA.W', 'AFB', 'AFG', 'AFGB', 'AFGC', 'AFGD', 'AFGE', 'AFIF', 'AFK', 'AFL', 'AFLG', 'AFMC', 'AFSM', 'AFT', 'AFTR', 'AFTR.U', 'AFTR.W', 'AFTY', 'AG', 'AGAC', 'AGAC.U', 'AGCO', 'AGD', 'AGE', 'AGG', 'AGGH', 'AGGY', 'AGI', 'AGIH', 'AGL', 'AGM', 'AGM$C', 'AGM$D', 'AGM